In [1]:
import pandas as pd
import pickle

In [2]:
palabras_lemm = 'palabras.pickle'
with open(palabras_lemm, "rb") as f:
    df_lemmas = pickle.load(f)

In [3]:
df_agrupado = df_lemmas.groupby(['lemma', 'hablante', 'grupo parlamentario', 'fecha', 'tipo palabra'])['palabra'].apply(list).reset_index()
df_agrupado['num_palabras'] = df_agrupado['palabra'].apply(len)
df_agrupado['palabras agrupadas'] = df_agrupado['palabra'].apply(lambda x: list(dict.fromkeys(x)))
df_agrupado.columns
#

Index(['lemma', 'hablante', 'grupo parlamentario', 'fecha', 'tipo palabra',
       'palabra', 'num_palabras', 'palabras agrupadas'],
      dtype='object')

In [4]:
#Seleccionamos las 20 palabras más dichas para elminarls
df_grouped = df_agrupado.groupby(['lemma'])['num_palabras'].sum().reset_index()
df_grouped.sort_values(by='num_palabras', ascending=False, inplace=True)
top_20_lemmas = df_grouped.head(20)
df_agrupado = df_agrupado[~df_agrupado['lemma'].isin(top_20_lemmas['lemma'])]
#las mpalabras más dichas
num_pal = 40
df_grouped = df_agrupado.groupby(['lemma'])['num_palabras'].sum().reset_index()
df_grouped.sort_values(by='num_palabras', ascending=False, inplace=True)
top_lemmas = df_grouped.head(num_pal)
#
df_grouped = df_agrupado[df_agrupado['lemma'].isin(top_lemmas['lemma'].tolist())]
grouped_by_lemma_and_group = df_grouped.groupby(['lemma', 'grupo parlamentario']).size().reset_index(name='count')
grouped_by_lemma_and_group = grouped_by_lemma_and_group.merge(top_lemmas, on='lemma', how='left')
grouped_by_lemma_and_group.rename(columns={'lemma': 'palabra', 'count': 'veces', 'num_palabras': 'total'}, inplace=True)
df_todos = grouped_by_lemma_and_group
#
df_grouped = df_agrupado.groupby(['lemma', 'fecha'])['num_palabras'].sum().reset_index()
df_resultado = pd.DataFrame()
dfs_top = [df_fecha.nlargest(num_pal, "num_palabras") for fecha, df_fecha in df_grouped.groupby("fecha")]
df_resultado = pd.concat(dfs_top)
top_lemmas = df_resultado
#
df_grouped =  pd.merge(df_agrupado, top_lemmas, on=["lemma", "fecha"], how="inner").reset_index()
df_grouped.rename(columns={'grupo parlamentario_x': 'grupo parlamentario'}, inplace=True)
grouped_by_lemma_and_group = df_grouped.groupby(['lemma', 'fecha', 'grupo parlamentario']).size().reset_index(name='count')
grouped_by_lemma_and_group = grouped_by_lemma_and_group.merge(top_lemmas, on='lemma', how='left')
grouped_by_lemma_and_group.rename(columns={'palabra': 'palabras','lemma': 'palabra', 'count': 'veces', 'num_palabras': 'total', 'grupo parlamentario_x': 'grupo parlamentario', 'fecha_x':'fecha'}, inplace=True)
grouped_by_lemma_and_group = grouped_by_lemma_and_group.drop(columns=['fecha_y'])
df_fecha = grouped_by_lemma_and_group
#
df_fecha.head()
#

,palabra,fecha,grupo parlamentario,veces,total
0,abandono,2023-02-09,Grupo Parlamentario Confederal de Unidas Podem...,2,23
1,abandono,2023-02-09,Grupo Parlamentario Euskal Herria Bildu,1,23
2,abandono,2023-02-09,Grupo Parlamentario Mixto,1,23
3,abandono,2023-02-09,Grupo Parlamentario Plural,1,23
4,abandono,2023-02-09,Grupo Parlamentario Popular en el Congreso,1,23


In [5]:
import networkx as nx
from bokeh.io import show, output_file, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import  NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx

output_notebook()
G = nx.Graph()
df_proceso = df_todos

for index, row in df_proceso.iterrows():
    G.add_node(row["grupo parlamentario"])
    G.add_node(row["palabra"])
    G.add_edge(row["grupo parlamentario"], row["palabra"], weight=row["total"])

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size='size', fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_color="black", line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Nombre", "@index"), ("Tamaño", "@size")])
plot.add_tools(hover, TapTool(), BoxSelectTool())

show(plot)


Loading BokehJS ...

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "size" value "size" [renderer: GlyphRenderer(id='1011', ...)]


In [6]:
import networkx as nx
from bokeh.io import show, output_file, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import  NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx

output_notebook()

G = nx.Graph()
df_proceso = df_todos

for index, row in df_proceso.iterrows():
    G.add_node(row["grupo parlamentario"], size=20)
    G.add_node(row["palabra"], size=20)
    G.add_edge(row["grupo parlamentario"], row["palabra"], weight=row["total"])

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='blue')
graph_renderer.edge_renderer.glyph = MultiLine(line_color="black", line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Nombre", "@index"), ("Tamaño", "@size")])
plot.add_tools(hover, TapTool(), BoxSelectTool())

show(plot)


Loading BokehJS ...

In [7]:
import networkx as nx
from bokeh.io import show, output_file, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np

output_notebook()
G = nx.Graph()

df_proceso = df_todos

# Normalizar valores de la columna "total"
min_total = df_proceso["total"].min()
max_total = df_proceso["total"].max()
df_proceso["total_normalizado"] = (df_proceso["total"] - min_total) / (max_total - min_total)

for index, row in df_proceso.iterrows():
    G.add_node(row["grupo parlamentario"], size=20)
    G.add_node(row["palabra"], size=20 + 30 * row["total_normalizado"])
    G.add_edge(row["grupo parlamentario"], row["palabra"], weight=row["total"])

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='blue')
graph_renderer.edge_renderer.glyph = MultiLine(line_color="black", line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Nombre", "@index"), ("Tamaño", "@size")])
plot.add_tools(hover, TapTool(), BoxSelectTool())

show(plot)


Loading BokehJS ...

In [8]:
import networkx as nx
from bokeh.io import show, output_file, output_notebook
from bokeh.io import output_notebook, save
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import  NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np
output_notebook()

G = nx.Graph()

df_proceso = df_todos

def normalize_size(value, max_value):
    return np.interp(value, [0, max_value], [1, 50])

max_total = df_proceso["veces"].max()

for index, row in df_proceso.iterrows():
    if row["grupo parlamentario"] not in G.nodes:
        G.add_node(row["grupo parlamentario"], size=20)
    if row["palabra"] not in G.nodes:
        G.add_node(row["palabra"], size=normalize_size(row["total"], max_total))
    G.add_edge(row["grupo parlamentario"], row["palabra"], weight=row["veces"])

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

graph_renderer = from_networkx(G, nx.spring_layout(G, k=10), scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='blue')
graph_renderer.edge_renderer.glyph = MultiLine(line_color="black", line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Nombre", "@index"), ("Tamaño", "@size")])
plot.add_tools(hover, TapTool(), BoxSelectTool())

show(plot)


Loading BokehJS ...

In [9]:
from bokeh.io import output_notebook, save
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np

output_notebook()

G = nx.Graph()

df_proceso = df_todos

def normalize_size(value, max_value):
    return np.interp(value, [0, max_value], [1, 30])

max_total = df_proceso["veces"].max()

for index, row in df_proceso.iterrows():
    if row["grupo parlamentario"] not in G.nodes:
        G.add_node(row["grupo parlamentario"], size=20, tipo="grupo_parlamentario", nombre=row["grupo parlamentario"])
    if row["palabra"] not in G.nodes:
        G.add_node(row["palabra"], size=normalize_size(row["total"], max_total), tipo="palabra", palabra=row["palabra"], total=row["total"])
    G.add_edge(row["grupo parlamentario"], row["palabra"], weight=row["veces"])

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))

graph_renderer = from_networkx(G, nx.spring_layout(G, k=12), scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='blue')
graph_renderer.edge_renderer.glyph = MultiLine(line_color="black", line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Tipo", "@tipo"),
                            ("Palabra", "@palabra"),
                            ("Nombre", "@nombre")])
plot.add_tools(hover, TapTool(), BoxSelectTool())


show(plot)


Loading BokehJS ...

In [10]:
from bokeh.io import output_notebook, save
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, ColumnDataSource
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np
from bokeh.models import LabelSet

output_notebook()

G = nx.Graph()

df_proceso = df_todos

def normalize_size(value, max_value):
    return np.interp(value, [0, max_value], [1, 50])

max_total = df_proceso["veces"].max()


for index, row in df_proceso.iterrows():
    if row["grupo parlamentario"] not in G.nodes:
        G.add_node(row["grupo parlamentario"], size=20, tipo="grupo_parlamentario", nombre=row["grupo parlamentario"], color='red')
    if row["palabra"] not in G.nodes:
        G.add_node(row["palabra"], size=normalize_size(row["total"], max_total), tipo="palabra", palabra=row["palabra"], total=row["total"], color='blue')
    G.add_edge(row["grupo parlamentario"], row["palabra"], weight=row["veces"])

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))

graph_renderer = from_networkx(G, nx.spring_layout(G, k=12), scale=1, center=(0, 0))

# Agrega nodos y enlaces al Plot
graph_renderer.node_renderer.data_source.data['fill_color'] = [node[1]['color'] for node in G.nodes(data=True)]
graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='fill_color')

graph_renderer.edge_renderer.glyph = MultiLine(line_color="black", line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Tipo", "@tipo"),
                            ("Palabra", "@palabra"),
                            ("Nombre", "@nombre")])
plot.add_tools(hover, TapTool(), BoxSelectTool())


#Add Labels
x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)


Loading BokehJS ...

In [11]:
colores_grupos = {
    'Grupo Parlamentario Ciudadanos':'#FF571F',
    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común':'#693279',
    'Grupo Parlamentario Popular en el Congreso': '#184A90',
    'Grupo Parlamentario Socialista': '#DC0612',
    'Grupo Parlamentario VOX': '#63BE21'
}
color_otro = '#ACB0B2'

In [12]:
from bokeh.io import output_notebook, save
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, ColumnDataSource
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np
from bokeh.models import LabelSet


output_notebook()

colores_grupos = {
    'Grupo Parlamentario Ciudadanos':'#FF571F',
    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común':'#693279',
    'Grupo Parlamentario Popular en el Congreso': '#184A90',
    'Grupo Parlamentario Socialista': '#DC0612',
    'Grupo Parlamentario VOX': '#63BE21'
}
color_otro = '#ACB0B2'

G = nx.Graph()

df_proceso = df_todos

def normalize_size(value, max_value):
    return np.interp(value, [0, max_value], [1, 50])

max_total = df_proceso["veces"].max()


for index, row in df_proceso.iterrows():
    if row["grupo parlamentario"] not in G.nodes:
        G.add_node(row["grupo parlamentario"], size=20, tipo="grupo_parlamentario", nombre=row["grupo parlamentario"], color=colores_grupos.get(row["grupo parlamentario"], color_otro))
    if row["palabra"] not in G.nodes:
        G.add_node(row["palabra"], size=normalize_size(row["total"], max_total), tipo="palabra", palabra=row["palabra"], total=row["total"], color='grey')
    G.add_edge(row["grupo parlamentario"], row["palabra"], weight=row["veces"], color=colores_grupos.get(row["grupo parlamentario"], color_otro))

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))

graph_renderer = from_networkx(G, nx.spring_layout(G, k=12), scale=1, center=(0, 0))

# Agrega nodos y enlaces al Plot
graph_renderer.node_renderer.data_source.data['fill_color'] = [node[1]['color'] for node in G.nodes(data=True)]
graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='fill_color')

graph_renderer.edge_renderer.glyph = MultiLine(line_color="black", line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Tipo", "@tipo"),
                            ("Palabra", "@palabra"),
                            ("Nombre", "@nombre")])
plot.add_tools(hover, TapTool(), BoxSelectTool())


#Add Labels
x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)

Loading BokehJS ...

In [13]:
from bokeh.io import output_notebook, save
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, ColumnDataSource
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np
from bokeh.models import LabelSet

output_notebook()

colores_grupos = {
    'Grupo Parlamentario Ciudadanos':'#FF571F',
    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común':'#693279',
    'Grupo Parlamentario Popular en el Congreso': '#184A90',
    'Grupo Parlamentario Socialista': '#DC0612',
    'Grupo Parlamentario VOX': '#63BE21'
}
color_otro = '#ACB0B2'


G = nx.Graph()

df_proceso = df_todos

def normalize_size(value, max_value):
    return np.interp(value, [0, max_value], [1, 50])

max_total = df_proceso["total"].max()


for index, row in df_proceso.iterrows():
    if row["grupo parlamentario"] not in G.nodes:
        G.add_node(row["grupo parlamentario"], size=20, tipo="grupo_parlamentario", nombre=row["grupo parlamentario"], color=colores_grupos.get(row["grupo parlamentario"], color_otro))
    if row["palabra"] not in G.nodes:
        G.add_node(row["palabra"], size=normalize_size(row["total"], max_total), tipo="palabra", palabra=row["palabra"], total=row["total"], color='white')
    G.add_edge(row["grupo parlamentario"], row["palabra"], weight=row["veces"], color=colores_grupos.get(row["grupo parlamentario"], color_otro))

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))

graph_renderer = from_networkx(G, nx.spring_layout(G, k=7), scale=1, center=(0, 0))

# Agrega nodos y enlaces al Plot
graph_renderer.node_renderer.data_source.data['fill_color'] = [node[1]['color'] for node in G.nodes(data=True)]
graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='fill_color')

def make_line_width(weight):
    return np.interp(weight, [0, max_total], [1, 10])

graph_renderer.edge_renderer.data_source.data['line_width'] = [make_line_width(edge[2]['weight']) for edge in G.edges(data=True)]
graph_renderer.edge_renderer.glyph = MultiLine(line_color='color', line_alpha=0.4, line_width='line_width')

plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Tipo", "@tipo"),
                            ("Palabra", "@palabra"),
                            ("Nombre", "@nombre")])
plot.add_tools(hover, TapTool(), BoxSelectTool())


#Add Labels
x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)



Loading BokehJS ...

In [19]:
from bokeh.io import output_notebook, save
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, ColumnDataSource
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np
from bokeh.models import LabelSet

output_notebook()

colores_grupos = {
    'Grupo Parlamentario Ciudadanos':'#FF571F',
    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común':'#693279',
    'Grupo Parlamentario Popular en el Congreso': '#184A90',
    'Grupo Parlamentario Socialista': '#DC0612',
    'Grupo Parlamentario VOX': '#63BE21'
}
color_otro = '#ACB0B2'


G = nx.Graph()

df_proceso = df_fecha[df_fecha['fecha'] == '2023-02-09']



def normalize_size(value, max_value):
    return np.interp(value, [0, max_value], [1, 50])

max_total = df_proceso.groupby(['palabra'])['veces'].sum().reset_index()['veces'].max()
max_veces = df_proceso['veces'].max()
dict_veces = df_proceso.groupby(['palabra'])['veces'].sum().reset_index().set_index('palabra')['veces'].to_dict()


for index, row in df_proceso.iterrows():
    if row["grupo parlamentario"] not in G.nodes:
        G.add_node(row["grupo parlamentario"], size=20, tipo="grupo_parlamentario", nombre=row["grupo parlamentario"], color=colores_grupos.get(row["grupo parlamentario"], color_otro))
    if row["palabra"] not in G.nodes:
        G.add_node(row["palabra"], size=normalize_size(dict_veces[row["palabra"]], max_total), tipo="palabra", palabra=row["palabra"], total=row["total"], color='white')
    G.add_edge(row["grupo parlamentario"], row["palabra"], weight=row["veces"], color=colores_grupos.get(row["grupo parlamentario"], color_otro))

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))

graph_renderer = from_networkx(G, nx.spring_layout(G, k=4), scale=1, center=(0, 0))

# Agrega nodos y enlaces al Plot
graph_renderer.node_renderer.data_source.data['fill_color'] = [node[1]['color'] for node in G.nodes(data=True)]
graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='fill_color')

def make_line_width(weight, max_value):
    return np.interp(weight, [0, max_value], [1, 5])

graph_renderer.edge_renderer.data_source.data['line_width'] = [make_line_width(edge[2]['weight'], max_veces) for edge in G.edges(data=True)]
graph_renderer.edge_renderer.glyph = MultiLine(line_color='color', line_alpha=0.7, line_width='line_width')

plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Tipo", "@tipo"),
                            ("Palabra", "@palabra"),
                            ("Nombre", "@nombre")])
plot.add_tools(hover, TapTool(), BoxSelectTool())


#Add Labels
x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
print(df_proceso.groupby(['palabra'])['veces'].sum().reset_index().sort_values(by='veces', ascending=False, inplace=False).head(40))
#

Loading BokehJS ...

          palabra  veces
30          poder   2988
17        gracias   2831
19         hablar   2744
16         gracia   2256
4            caso   2015
39            ver   1512
6       comunidad   1476
21     importante   1248
14       enmienda   1156
11          deber   1079
36       señorías    972
23         llegar    910
18          haber    896
38        trabajo    812
8           creer    728
33       proyecto    696
12          dejar    672
9        cuestión    610
24          lugar    552
37         tiempo    500
20          hacer    364
15       entender    330
31     protección    289
28          penal    255
22  internacional    238
2          animal    174
26        materia    121
35         quedar    112
1          acabar    100
27          mundo     84
10         código     77
7     cooperación     68
3       bienestar     65
32       proteger     45
5            caza     42
13     desarrollo     40
29          perro     28
25       maltrato     24
34           psoe     14


In [45]:
list(df_todos['grupo parlamentario'].unique())

['Defensor del Pueblo',
 'Defensor del Pueblo en Funciones',
 'Grupo Parlamentario Ciudadanos',
 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común',
 'Grupo Parlamentario Euskal Herria Bildu',
 'Grupo Parlamentario Mixto',
 'Grupo Parlamentario Plural',
 'Grupo Parlamentario Popular en el Congreso',
 'Grupo Parlamentario Republicano',
 'Grupo Parlamentario Socialista',
 'Grupo Parlamentario VOX',
 'Grupo Parlamentario Vasco (EAJ-PNV)',
 'Junta General del Principado de Asturias',
 'Parlamento de Andalucía',
 'Parlamento de Cantabria',
 'Parlamento de Galicia',
 'Parlamento de Illes Balears',
 'Parlamento de La Rioja',
 'Parlamento de Murcia',
 'Parlamento de Pais Vasco',
 'Parlamento del principado de Asturias',
 'Parlamento del principado de Canarias',
 'Parlamento de Aragón',
 'Parlamento de Cataluña',
 'Parlamento de Navarra',
 'Asamblea de Madrid']

In [16]:
df_proceso.groupby(['palabra'])['veces'].sum().reset_index()['veces'].max()

4619

In [23]:
df_fecha.head()

,palabra,fecha,grupo parlamentario,veces,total
0,abandono,2023-02-09,Grupo Parlamentario Confederal de Unidas Podem...,2,23
1,abandono,2023-02-09,Grupo Parlamentario Euskal Herria Bildu,1,23
2,abandono,2023-02-09,Grupo Parlamentario Mixto,1,23
3,abandono,2023-02-09,Grupo Parlamentario Plural,1,23
4,abandono,2023-02-09,Grupo Parlamentario Popular en el Congreso,1,23


In [17]:
df_fecha['fecha'].unique().tolist()

['2023-02-09',
 '2020-01-04',
 '2020-10-21',
 '2020-10-22',
 '2021-02-16',
 '2021-09-21',
 '2022-10-06',
 '2021-10-19',
 '2022-03-08',
 '2020-07-29',
 '2020-09-15',
 '2020-09-23',
 '2020-11-12',
 '2021-04-22',
 '2021-06-15',
 '2021-06-16',
 '2022-03-15',
 '2022-06-07',
 '2022-03-17',
 '2022-12-13',
 '2022-06-23',
 '2021-05-13',
 '2020-06-11',
 '2020-01-07',
 '2022-04-06',
 '2021-05-18',
 '2020-04-09',
 '2020-04-15',
 '2020-05-13',
 '2020-05-27',
 '2020-06-10',
 '2020-07-15',
 '2020-09-29',
 '2021-03-25',
 '2021-05-25',
 '2021-10-13',
 '2022-04-07',
 '2022-12-22',
 '2023-02-07',
 '2022-09-20',
 '2022-09-21',
 '2021-12-28',
 '2020-02-26',
 '2020-02-27',
 '2020-06-25',
 '2020-09-10',
 '2020-10-13',
 '2020-10-20',
 '2020-11-17',
 '2020-12-15',
 '2021-02-25',
 '2021-06-23',
 '2021-07-21',
 '2021-09-14',
 '2021-09-23',
 '2021-12-02',
 '2021-12-23',
 '2022-04-05',
 '2022-06-30',
 '2022-09-22',
 '2022-09-28',
 '2022-10-04',
 '2022-11-22',
 '2022-12-14',
 '2022-12-20',
 '2021-04-15',
 '2020-02-

In [18]:
list(set(df_todos['palabra'].tolist()))

['importante',
 'señorías',
 'gracia',
 'gracias',
 'presupuesto',
 'enmienda',
 'tiempo',
 'haber',
 'europeo',
 'deber',
 'presidente',
 'cosa',
 'pedir',
 'autónomo',
 'poner',
 'seguir',
 'momento',
 'tener',
 'llegar',
 'empresa',
 'sistema',
 'cámara',
 'sector',
 'trabajo',
 'venir',
 'euros',
 'socialista',
 'hablar',
 'situación',
 'popular',
 'crisis',
 'parlamentario',
 'ver',
 'caso',
 'económico',
 'comunidad',
 'político',
 'real',
 'poder',
 'problema']

In [20]:
list(set(df_todos['grupo parlamentario'].tolist()))

['Parlamento de Cataluña',
 'Grupo Parlamentario Republicano',
 'Parlamento del principado de Asturias',
 'Grupo Parlamentario Euskal Herria Bildu',
 'Parlamento de Navarra',
 'Parlamento de La Rioja',
 'Parlamento de Murcia',
 'Parlamento de Andalucía',
 'Grupo Parlamentario Mixto',
 'Parlamento del principado de Canarias',
 'Grupo Parlamentario Popular en el Congreso',
 'Parlamento de Illes Balears',
 'Asamblea de Madrid',
 'Parlamento de Aragón',
 'Defensor del Pueblo',
 'Defensor del Pueblo en Funciones',
 'Parlamento de Pais Vasco',
 'Grupo Parlamentario Plural',
 'Parlamento de Cantabria',
 'Parlamento de Galicia',
 'Grupo Parlamentario Ciudadanos',
 'Grupo Parlamentario Vasco (EAJ-PNV)',
 'Grupo Parlamentario VOX',
 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común',
 'Grupo Parlamentario Socialista',
 'Junta General del Principado de Asturias']

In [21]:
from py2neo import Graph

In [22]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "congreso"))

In [25]:
df_proceso = graph.run("CALL { \
MATCH (h:Hablante)-[d:DICE]->(p:Palabra) \
WHERE d.fecha = '2023-02-09' \
WITH p.lemma as palabra, p.pagerank_DICE_2023_02_09 as pg, sum(d.veces) as total \
RETURN DISTINCT(palabra) as palabra, pg, total \
order by pg desc \
LIMIT 40 \
} WITH palabra, total \
MATCH (h:Hablante)-[d:DICE]->(p:Palabra) \
WHERE d.fecha = '2023-02-09' \
AND p.lemma = palabra \
RETURN h.grupo_parlamentario as grupo_parlamentario, sum(d.veces) as veces, p.lemma as palabra, d.fecha as fecha, total").to_data_frame()

In [26]:
df_proceso

,grupo_parlamentario,veces,palabra,fecha,total
0,Grupo Parlamentario Euskal Herria Bildu,15,animal,2023-02-09,176
1,Grupo Parlamentario Plural,22,animal,2023-02-09,176
2,Grupo Parlamentario Mixto,10,animal,2023-02-09,176
3,Grupo Parlamentario Socialista,11,animal,2023-02-09,176
4,Grupo Parlamentario Popular en el Congreso,12,animal,2023-02-09,176
...,...,...,...,...,...
308,Grupo Parlamentario Socialista,3,comisión,2023-02-09,17
309,Grupo Parlamentario Confederal de Unidas Podem...,2,comisión,2023-02-09,17
310,Grupo Parlamentario Mixto,1,comisión,2023-02-09,17
311,Grupo Parlamentario Ciudadanos,1,comisión,2023-02-09,17


In [42]:
from bokeh.io import output_notebook, save
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, ColumnDataSource
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np
from bokeh.models import LabelSet
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "congreso"))

output_notebook()

colores_grupos = {
    'Grupo Parlamentario Ciudadanos':'#FF571F',
    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común':'#693279',
    'Grupo Parlamentario Popular en el Congreso': '#184A90',
    'Grupo Parlamentario Socialista': '#DC0612',
    'Grupo Parlamentario VOX': '#63BE21'
}
color_otro = '#ACB0B2'


G = nx.Graph()

df_proceso = graph.run("CALL { \
MATCH (h:Hablante)-[d:DICE]->(p:Palabra) \
WHERE d.fecha = $fecha \
WITH p.lemma as palabra, p.pagerank_DICE_2023_02_09 as pg \
RETURN DISTINCT(palabra) as palabra, pg \
order by pg desc \
LIMIT 40 \
} WITH palabra, pg \
MATCH (h:Hablante)-[d:DICE]->(p:Palabra) \
WHERE d.fecha = $fecha \
AND p.lemma = palabra \
RETURN h.grupo_parlamentario as grupo_parlamentario, sum(d.veces) as veces, \
p.lemma as palabra, d.fecha as fecha, pg as total",
                      fecha='2023-02-09').to_data_frame()



def normalize_size(value, max_value):
    return np.interp(value, [0, max_value], [1, 50])

max_total = df_proceso['total'].max() * 100
max_veces = df_proceso['veces'].max()
dict_veces = df_proceso.groupby(['palabra'])['veces'].sum().reset_index().set_index('palabra')['veces'].to_dict()


for index, row in df_proceso.iterrows():
    if row["grupo_parlamentario"] not in G.nodes:
        G.add_node(row["grupo_parlamentario"], size=20, tipo="grupo_parlamentario", nombre=row["grupo_parlamentario"], color=colores_grupos.get(row["grupo_parlamentario"], color_otro))
    if row["palabra"] not in G.nodes:
        G.add_node(row["palabra"], size=normalize_size(dict_veces[row["palabra"]], max_total), tipo="palabra", palabra=row["palabra"], total=row["total"], color='white')
    G.add_edge(row["grupo_parlamentario"], row["palabra"], weight=row["veces"], color=colores_grupos.get(row["grupo_parlamentario"], color_otro))

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title= 'Palabras más importantes en 2023-02-09')

graph_renderer = from_networkx(G, nx.spring_layout(G, k=4), scale=1, center=(0, 0))

# Agrega nodos y enlaces al Plot
graph_renderer.node_renderer.data_source.data['fill_color'] = [node[1]['color'] for node in G.nodes(data=True)]
graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='fill_color')

def make_line_width(weight, max_value):
    return np.interp(weight, [0, max_value], [1, 5])

graph_renderer.edge_renderer.data_source.data['line_width'] = [make_line_width(edge[2]['weight'], max_veces) for edge in G.edges(data=True)]
graph_renderer.edge_renderer.glyph = MultiLine(line_color='color', line_alpha=0.7, line_width='line_width')

plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Tipo", "@tipo"),
                            ("Palabra", "@palabra"),
                            ("Nombre", "@nombre")])
plot.add_tools(hover, TapTool(), BoxSelectTool())


#Add Labels
x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
print(df_proceso.groupby(['palabra'])['veces'].sum().reset_index().sort_values(by='veces', ascending=False, inplace=False).head(40))
#

Loading BokehJS ...

          palabra  veces
1          animal    176
8     cooperación    103
31          perro     68
15       enmienda     49
4            caza     42
30          penal     40
11         código     37
34     protección     35
29          mundo     31
20         hablar     31
13          dejar     29
18        gracias     28
9           creer     27
17         gracia     27
2       bienestar     25
25       maltrato     25
22     importante     25
23  internacional     25
24          lugar     24
37         quedar     24
38        trabajo     24
32          poder     23
35       proyecto     23
26        materia     23
0        abandono     23
19          haber     23
16       entender     23
12          deber     22
3            caso     21
14     desarrollo     20
10       cuestión     20
36           psoe     20
21          hacer     19
6       comunidad     18
39          votar     18
27        mayoría     17
5        comisión     17
7        convenio     15
33       político     14


In [41]:
from bokeh.io import output_notebook, save
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, ColumnDataSource
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np
from bokeh.models import LabelSet
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "congreso"))

output_notebook()

colores_grupos = {
    'Grupo Parlamentario Ciudadanos':'#FF571F',
    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común':'#693279',
    'Grupo Parlamentario Popular en el Congreso': '#184A90',
    'Grupo Parlamentario Socialista': '#DC0612',
    'Grupo Parlamentario VOX': '#63BE21'
}
color_otro = '#ACB0B2'


G = nx.Graph()

df_proceso = graph.run("CALL { \
MATCH (h:Hablante)-[d:DICE]->(p:Palabra) \
WHERE d.fecha = $fecha \
AND h.grupo_parlamentario = $grupo \
WITH p.lemma as palabra, p.pagerank_DICE_2023_02_09_VOX as pg \
RETURN DISTINCT(palabra) as palabra, pg \
order by pg desc \
LIMIT 40 \
} WITH palabra, pg \
MATCH (h:Hablante)-[d:DICE]->(p:Palabra) \
WHERE d.fecha = $fecha \
AND p.lemma = palabra \
AND h.grupo_parlamentario = $grupo \
RETURN h.grupo_parlamentario as grupo_parlamentario, sum(d.veces) as veces, \
p.lemma as palabra, d.fecha as fecha, pg as total",
                      fecha='2023-02-09', 
                       grupo = 'Grupo Parlamentario VOX').to_data_frame()



def normalize_size(value, max_value):
    return np.interp(value, [0, max_value], [1, 50])

max_total = df_proceso['total'].max() * 100
max_veces = df_proceso['veces'].max()
dict_veces = df_proceso.groupby(['palabra'])['veces'].sum().reset_index().set_index('palabra')['veces'].to_dict()


for index, row in df_proceso.iterrows():
    if row["grupo_parlamentario"] not in G.nodes:
        G.add_node(row["grupo_parlamentario"], size=20, tipo="grupo_parlamentario", nombre=row["grupo_parlamentario"], color=colores_grupos.get(row["grupo_parlamentario"], color_otro))
    if row["palabra"] not in G.nodes:
        G.add_node(row["palabra"], size=normalize_size(dict_veces[row["palabra"]], max_total), tipo="palabra", palabra=row["palabra"], total=row["total"], color='white')
    G.add_edge(row["grupo_parlamentario"], row["palabra"], weight=row["veces"], color=colores_grupos.get(row["grupo_parlamentario"], color_otro))

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title= 'Palabras más importantes de Grupo Parlamentario VOX en 2023-02-09')

graph_renderer = from_networkx(G, nx.spring_layout(G, k=4), scale=1, center=(0, 0))

# Agrega nodos y enlaces al Plot
graph_renderer.node_renderer.data_source.data['fill_color'] = [node[1]['color'] for node in G.nodes(data=True)]
graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='fill_color')

def make_line_width(weight, max_value):
    return np.interp(weight, [0, max_value], [1, 5])

graph_renderer.edge_renderer.data_source.data['line_width'] = [make_line_width(edge[2]['weight'], max_veces) for edge in G.edges(data=True)]
graph_renderer.edge_renderer.glyph = MultiLine(line_color='color', line_alpha=0.7, line_width='line_width')

plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Tipo", "@tipo"),
                            ("Palabra", "@palabra"),
                            ("Nombre", "@nombre")])
plot.add_tools(hover, TapTool(), BoxSelectTool())


#Add Labels
x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
print(df_proceso.groupby(['palabra'])['veces'].sum().reset_index().sort_values(by='veces', ascending=False, inplace=False).head(40))
#

Loading BokehJS ...

        palabra  veces
2        animal     22
27        mundo      9
7   cooperación      9
20       hombre      8
14        gente      8
26        miedo      7
6      convenio      7
19       hambre      7
11     enmienda      6
34        reino      6
12     escuchar      6
0        acabar      6
22       llegar      5
9         deber      5
15      gracias      5
17       hablar      5
30        perro      5
8         creer      5
29      oficina      5
3        ayudar      5
33        razón      5
35          sed      5
13         frío      5
36      terreno      4
31    pretender      4
39        venir      4
24        lugar      4
23        lucha      4
21  humanitario      4
37   terrorismo      4
18        hacer      4
38       unidas      4
32   protección      4
10   desarrollo      3
28     naciones      3
4          caso      3
25      materia      3
1        agenda      3
16        haber      3
5        centro      3


In [71]:
fechas = df_lemmas['fecha'].unique()
grupos = df_lemmas['grupo parlamentario'].unique()

In [81]:
df_fechas = pd.DataFrame(df_lemmas['fecha'].unique(), columns=['fecha'])
df_grupos = pd.DataFrame(df_lemmas['grupo parlamentario'].unique(), columns=['grupo parlamentario'])

In [107]:
(len(df_fechas) *2) + 2 + (len(df_grupos) * 2 ) + (len(df_resultado) * 2)

12582

In [112]:
seg = 600
cuantos = 240
(((((len(df_fechas) *2) + 2 + (len(df_grupos) * 2 ) + (len(df_resultado) * 2)) * seg) / cuantos) / 60) / 60

8.7375

In [83]:
df_resultado = pd.merge(df_fechas, df_grupos, how='cross')
df_resultado

,fecha,grupo parlamentario
0,2020-02-19,Grupo Parlamentario Popular en el Congreso
1,2020-02-19,Grupo Parlamentario Socialista
2,2020-02-19,Grupo Parlamentario Ciudadanos
3,2020-02-19,Grupo Parlamentario VOX
4,2020-02-19,Grupo Parlamentario Confederal de Unidas Podem...
...,...,...
6027,2021-05-11,Parlamento de Galicia
6028,2021-05-11,Parlamento de Illes Balears
6029,2021-05-11,Parlamento de Pais Vasco
6030,2021-05-11,Parlamento de La Rioja


In [88]:
df = graph.run(f"CALL gds.graph.project.cypher( \
          'DISCURSO_{fecha}', \
          'MATCH (n) RETURN id(n) AS id, labels(n) as labels', \
          'MATCH (p1:Palabra)-[d:DISCURSO]->(p2:Palabra) \
          WHERE d.fecha = \"{fecha} \" \
           RETURN id(p1) AS source, id(p2) AS target, type(d) AS type') \
          YIELD graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels").to_data_frame()

In [92]:
dict_abreviaturas_grupos = {
 'Grupo Parlamentario Ciudadanos': 'GCs',
 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común': 'GCUP-EC-GC',
 'Grupo Parlamentario Euskal Herria Bildu':'GEH-Bildu',
 'Grupo Parlamentario Mixto': 'GMx',
 'Grupo Parlamentario Plural': 'GPlu',
 'Grupo Parlamentario Popular en el Congreso': 'GP',
 'Grupo Parlamentario Republicano': 'GR',
 'Grupo Parlamentario Socialista': 'GS',
 'Grupo Parlamentario VOX': 'GVOX',
 'Grupo Parlamentario Vasco (EAJ-PNV)': 'GV-(EAJ-PNV)',
}

In [95]:
buscar_abreviacion = lambda x: dict_abreviaturas_grupos.get(x, 'N/A')
df_grupos['abreb'] = df_grupos['grupo parlamentario'].apply(buscar_abreviacion)

In [96]:
df_grupos

,grupo parlamentario,abreb
0,Grupo Parlamentario Popular en el Congreso,GP
1,Grupo Parlamentario Socialista,GS
2,Grupo Parlamentario Ciudadanos,GCs
3,Grupo Parlamentario VOX,GVOX
4,Grupo Parlamentario Confederal de Unidas Podem...,GCUP-EC-GC
5,Grupo Parlamentario Mixto,GMx
6,Grupo Parlamentario Plural,GPlu
7,Grupo Parlamentario Republicano,GR
8,Grupo Parlamentario Vasco (EAJ-PNV),GV-(EAJ-PNV)
9,Grupo Parlamentario Euskal Herria Bildu,GEH-Bildu


In [102]:
graph.run("call gds.graph.list() YIELD graphName RETURN COUNT(graphName) as numero").to_series()[0]

13

In [114]:
'False'

True